In [1]:
import warnings
warnings.filterwarnings('ignore')
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


학습 데이터를 만든다.

In [2]:
xData = [1, 2, 3, 4, 5, 6, 7] # 근무 시간
yData = [25000, 55000, 75000, 110000, 128000, 155000, 180000] # 근무 시간에 따른 매출 금액 => 실제 관찰된 값 => 실제값

난수를 발생시켜 학습할 데이터의 기울기(가중치)와 y절편(바이어스)를 정한다.  
난수를 발생시켜 작업하는 이유는 과적합을 방지하기 위해서이다.  
과적합이란 학습에 사용한 데이터에서는 높은 정확도를 보이지만 학습시킨 데이터 이외의 데이터에서는 낮은 정확도를 보이는 문제점을 말한다.

In [3]:
# random_uniform([난수의 개수], 난수의 최소값, 난수의 최대값) 함수로 균등분포로부터의 난수값을 발생시킨다.
a = tf.Variable(tf.random_uniform([1], -100, 100)) # 기울기(가중치), -100 ~ 100 사이의 난수
b = tf.Variable(tf.random_uniform([1], -100, 100)) # y절편(바이어스), -100 ~ 100 사이의 난수
print('a: {}, b: {}'.format(a, b))

a: <tf.Variable 'Variable:0' shape=(1,) dtype=float32_ref>, b: <tf.Variable 'Variable_1:0' shape=(1,) dtype=float32_ref>


In [4]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
print('a: {}, b: {}'.format(sess.run(a), sess.run(b)))

a: [44.32666], b: [76.62166]


근무 시간과 매출 금액을 기억할 placeholder를 선언한다.

In [5]:
x = tf.placeholder(dtype=tf.float32) # 근무 시간(xData)를 기억하는 placeholder
y = tf.placeholder(dtype=tf.float32) # 매출 금액(yData)를 기억하는 placeholder

1차 방정식 형태의 가설을 세우고 오차(비용) 함수를 정의한다.

In [10]:
# 1차 방정식 형태의 가설 Y(예측값)를 정의한다.
Y = a * x + b
# 비용(오차) 함수 => 실제값(yData => y라는 placeholder에 저장한다.)과 예측값(Y)의 편차의 제곱에 대한 평균을 이용한다.
# => 평균 제곱법
# reduce_mean() 함수로 tensorflow에서 평균을 계산한다.
# square() 함수로 tensorflow에서 제곱값을 계산한다.
# sqrt() 함수로 tensorflow에서 제곱근 값을 계산한다.
cost = tf.sqrt(tf.reduce_mean(tf.square(Y - y))) # 예측값과 실제값의 편차에 대한 평균의 제곱근을 계산한다.

경사 하강법을 이용해서 비용(오차) 함수의 결과를 가장 작게 만드는 방향으로 학습을 시키도록 정의한다.

In [33]:
# 경사 하강 알고리즘의 학습율을 설정한다.
learning_rate = 0.3
# train.GradientDescentOptimizer(학습율) 함수로 학습율에 따른 비용(오차) 함수를 계산한다.
# optimizer = tf.train.GradientDescentOptimizer(learning_rate)
# minimize() 함수로 경사 하강 알고리즘에 따라 인수로 지정된 비용(오차) 함수의 최소값을 찾는다.
# train = optimizer.minimize(cost)
train = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

학습시킨다.

In [34]:
# 세션을 만들과 변수를 초기화시킨다.
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(100001):
    # 비용(오차) 함수를 가장 작게 만드는 학습을 할 수 있도록 placeholder에 데이터를 대입한다.
    # 학습할 데이터는 반드시 'feed_dict'라는 이름의 딕셔너리 타입의 자료형을 만들어 대입해야 한다.
    # 'feed_dict' 딕셔너리의 key에는 placeholder의 이름을, value에는 각 placeholder에 대입할 리스트 타입의 데이터를 지정한다.
    
    # tensorflow 세션에서 비용(오차) 함수를 가장 작게 만드는 방향으로 'feed_dict' 딕셔너리의 데이터로 학습시킨다.
    sess.run(train, feed_dict={x: xData, y: yData})
    
    # 500 Epoch 단위로 학습이 될 때 마다 머신러닝이 실행되는 중간 결과를 확인한다.
    if i % 5000 == 0:
        # i, 오차, 기울기, y절편
        # print(i, sess.run(cost, feed_dict={x: xData, y: yData}), sess.run(a), sess.run(b))
        # print('%4d. %10.3f %7.3f %7.3f' % (i, sess.run(cost, feed_dict={x: xData, y: yData}), sess.run(a), sess.run(b)))
        print('{0:6d}. {1:10.3f} {2:9.3f} {3:9.3f}'.format(i, sess.run(cost, feed_dict={x: xData, y: yData}), sess.run(a)[0], 
                                                           sess.run(b)[0]))

     0. 116265.680   -53.465   -35.221
  5000.  85090.141  6651.565  1308.108
 10000.  53953.113 13353.639  2645.835
 15000.  23020.705 20036.191  3966.008
 20000.   3307.297 24923.896  4792.209
 25000.   3246.435 25019.629  4515.646
 30000.   3197.970 25072.561  4251.428
 35000.   3156.424 25121.926  4007.150
 40000.   3120.812 25167.881  3780.593
 45000.   3090.669 25209.572  3572.369
 50000.   3065.201 25248.736  3381.185
 55000.   3043.689 25284.100  3205.017
 60000.   3025.723 25316.266  3044.293
 65000.   3010.744 25345.906  2897.812
 70000.   2998.158 25373.516  2763.079
 75000.   2987.724 25398.053  2640.441
 80000.   2979.098 25420.318  2529.150
 85000.   2971.975 25440.533  2428.231
 90000.   2966.034 25459.934  2335.947
 95000.   2961.097 25476.850  2251.379
100000.   2957.038 25492.162  2174.821


In [35]:
print('a: {}, b: {}'.format(sess.run(a), sess.run(b)))

a: [25492.162], b: [2174.8213]


In [37]:
import time
for i in range(9, 25):
    print('{0:2d}시간 근무했을 때 매출 기대금액: {1:7,.0f}'.format(i, sess.run(Y, feed_dict={x: [i]})[0]))
    time.sleep(0.5)

 9시간 근무했을 때 매출 기대금액: 231,604
10시간 근무했을 때 매출 기대금액: 257,096
11시간 근무했을 때 매출 기대금액: 282,589
12시간 근무했을 때 매출 기대금액: 308,081
13시간 근무했을 때 매출 기대금액: 333,573
14시간 근무했을 때 매출 기대금액: 359,065
15시간 근무했을 때 매출 기대금액: 384,557
16시간 근무했을 때 매출 기대금액: 410,049
17시간 근무했을 때 매출 기대금액: 435,542
18시간 근무했을 때 매출 기대금액: 461,034
19시간 근무했을 때 매출 기대금액: 486,526
20시간 근무했을 때 매출 기대금액: 512,018
21시간 근무했을 때 매출 기대금액: 537,510
22시간 근무했을 때 매출 기대금액: 563,002
23시간 근무했을 때 매출 기대금액: 588,495
24시간 근무했을 때 매출 기대금액: 613,987
